In [ ]:
#r "nuget: Elastic.Clients.Elasticsearch"
#r "nuget: System.Net.Http.Json"
#!import ./Utils.cs
#!import ../_infra/get-connection-string.ipynb

using Elastic.Transport;
using Elastic.Clients.Elasticsearch;
using Elastic.Transport.Products.Elasticsearch;

var elasticSettings = new ElasticsearchClientSettings(connectionString)
    .DisableDirectStreaming()
    .ServerCertificateValidationCallback(CertificateValidations.AllowAll);

var client = new ElasticsearchClient(elasticSettings);

## Test the Client

Before you continue, confirm that the client has connected with this test.


In [ ]:
var info = await client.InfoAsync();

display(info);

## Index some test data
Our client is set up and connected to our Elastic deployment. Now we need some data to test out the basics of Elasticsearch queries. We'll use a small index of books with the following fields

In [ ]:
using System.Text.Json.Serialization;

public class Book
{
    [JsonPropertyName("title")]
    public string Title { get; set; }

    [JsonPropertyName("summary")]
    public string Summary { get; set; }

    [JsonPropertyName("publish_date")]
    public DateTimeOffset PublishDate { get; set; }

    [JsonPropertyName("num_reviews")]
    public int NumReviews { get; set; }

    [JsonPropertyName("publisher")]
    public string Publisher { get; set; }


    public object? TitleVector { get; set; }
}

In [ ]:
using Elastic.Clients.Elasticsearch;
using Elastic.Clients.Elasticsearch.IndexManagement;
using Elastic.Clients.Elasticsearch.Mapping;

const string BookIndex = "book_index";
var indexDescriptor = new CreateIndexRequestDescriptor<Book>(BookIndex)
    .Mappings(m => m
        .Properties(pp => pp
            .Text(p => p.Title)
            .DenseVector(
                Infer.Property<Book>(p => p.TitleVector),
                d => d.Dims(384).Index(true).Similarity(DenseVectorSimilarity.Cosine))
            .Text(p => p.Summary)
            .Date(p => p.PublishDate)
            .IntegerNumber(p => p.NumReviews)
            .Keyword(p => p.Publisher)
        )
    );

In [ ]:
var indexResponse = await client.Indices.CreateAsync(indexDescriptor);

display(indexResponse);
ToJson(DumpGetRequest(indexResponse.DebugInformation)).Display();

In [ ]:
using System.Net.Http;
using System.Net.Http.Json;

var http = new HttpClient();
var url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json";
var books =  await http.GetFromJsonAsync<Book[]>(url);

display(books);


In [ ]:
var bulkResponse = await client.BulkAsync(BookIndex, d => d
    .IndexMany<Book>(books, (bd, b) => bd.Index(BookIndex).));

display(DumpGetRequest(bulkResponse));

In [ ]:
var searchResponse = await client.SearchAsync<Book>(s => 
    s   
        .Index(BookIndex)
        .Query(q => q.Match(d => d.Field(f => f.Title).Query("JavaScript")))
);

DumpGetRequest(searchResponse).Display();

searchResponse.Display();